In [ ]:
pip install requests pandas

In [ ]:
import requests
import time
import hmac
import hashlib
import json
from datetime import datetime

In [ ]:
#Grid Trading

In [ ]:
# Binance US API credentials
api_key = 'YOUR_API_KEY'
secret_key = 'YOUR_SECRET_KEY'

In [ ]:
# Create the signature required by Binance US API
def generate_signature(query_string, secret_key):
    return hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

# Fetch the current market price from Binance US
def fetch_market_price(symbol):
    base_url = 'https://api.binance.us'
    endpoint = f'/api/v3/ticker/price?symbol={symbol}'
    url = base_url + endpoint

    response = requests.get(url)
    data = response.json()
    if 'price' in data:
        return float(data['price'])
    else:
        raise Exception(f"Error fetching data: {data}")

In [ ]:
# Example usage
symbol = 'BTCUSDT'
try:
    current_price = fetch_market_price(symbol)
    print(f"Current price of {symbol}: ${current_price}")
except Exception as e:
    print(e)

In [ ]:
# Implement Grid Trading
import random

#current_price: current price of designated cypto retrieved from market
#lower_bound: refers to when the price drops to a certain level, a buy order is executed
#upper_bound: refers to when the price rises to a certain level, a sell order is executed
#num_levels: determines how many price points will have active orders; more levels can mean more opportunities for trade.
#intervals: the intervals between prices levels (e.g., every $10 or every 1%)

def create_grid(current_price, lower_bound, upper_bound, num_levels):
    interval = (upper_bound - lower_bound) / num_levels
    buy_orders = []
    sell_orders = []

    for i in range(num_levels):
        buy_price = lower_bound + i * interval
        sell_price = upper_bound - i * interval
        if buy_price < current_price:
            buy_orders.append(buy_price)
        if sell_price > current_price:
            sell_orders.append(sell_price)

    return buy_orders, sell_orders

def simulate_trading(buy_orders, sell_orders, initial_balance, initial_coins, price_fluctuations):
    balance = initial_balance
    coins = initial_coins

    for price in price_fluctuations:
        # Execute buy orders
        for buy_price in buy_orders:
            if price <= buy_price and balance >= buy_price:
                coins += 1
                balance -= buy_price
                print(f"Bought 1 coin at ${buy_price:.2f}")

        # Execute sell orders
        for sell_price in sell_orders:
            if price >= sell_price and coins > 0:
                coins -= 1
                balance += sell_price
                print(f"Sold 1 coin at ${sell_price:.2f}")

    final_balance = balance + coins * price_fluctuations[-1]
    print(f"Final balance: ${final_balance:.2f}")
    return final_balance

In [ ]:
# Example usage
symbol = 'BTCUSDT'
try:
    current_price = fetch_market_price(symbol)
    print(f"Current price of {symbol}: ${current_price}")
    
    lower_bound = 30000
    upper_bound = 40000
    num_levels = 10

    buy_orders, sell_orders = create_grid(current_price, lower_bound, upper_bound, num_levels)
    print("Buy Orders:", buy_orders)
    print("Sell Orders:", sell_orders)

    initial_balance = 10000
    initial_coins = 0
    price_fluctuations = [current_price + random.uniform(-1000, 1000) for _ in range(30)]

    simulate_trading(buy_orders, sell_orders, initial_balance, initial_coins, price_fluctuations)
except Exception as e:
    print(e)

In [ ]:
# Place Real Orders on Binance US
def place_order(symbol, side, size, price):
    base_url = 'https://api.binance.us'
    endpoint = '/api/v3/order'
    url = base_url + endpoint

    timestamp = int(time.time() * 1000)
    params = {
        'symbol': symbol,
        'side': side.upper(),
        'type': 'LIMIT',
        'timeInForce': 'GTC',
        'quantity': size,
        'price': price,
        'recvWindow': 5000,
        'timestamp': timestamp
    }
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = generate_signature(query_string, secret_key)
    params['signature'] = signature

    headers = {
        'X-MBX-APIKEY': api_key
    }

    response = requests.post(url, headers=headers, params=params)
    data = response.json()
    if 'orderId' in data:
        return data
    else:
        raise Exception(f"Error placing order: {data}")

# Example usage
symbol = 'BTCUSDT'
side = 'buy'
size = 0.001  # size in BTC
price = 30000  # limit price
try:
    order = place_order(symbol, side, size, price)
    print(f"Order placed: {order}")
except Exception as e:
    print(e)

In [ ]:
# DCA trading

In [ ]:
# Binance US API credentials
api_key = 'YOUR_API_KEY'
secret_key = 'YOUR_SECRET_KEY'

In [ ]:
# Create the signature required by Binance US API
def generate_signature(query_string, secret_key):
    return hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

# Fetch the current market price from Binance US
def fetch_market_price(symbol):
    base_url = 'https://api.binance.us'
    endpoint = f'/api/v3/ticker/price?symbol={symbol}'
    url = base_url + endpoint

    response = requests.get(url)
    data = response.json()
    if 'price' in data:
        return float(data['price'])
    else:
        raise Exception(f"Error fetching data: {data}")

In [ ]:
# Place an order on Binance US
def place_order(symbol, side, size, price):
    base_url = 'https://api.binance.us'
    endpoint = '/api/v3/order'
    url = base_url + endpoint

    timestamp = int(time.time() * 1000)
    params = {
        'symbol': symbol,
        'side': side.upper(),
        'type': 'LIMIT',
        'timeInForce': 'GTC',
        'quantity': size,
        'price': price,
        'recvWindow': 5000,
        'timestamp': timestamp
    }
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = generate_signature(query_string, secret_key)
    params['signature'] = signature

    headers = {
        'X-MBX-APIKEY': api_key
    }

    response = requests.post(url, headers=headers, params=params)
    data = response.json()
    if 'orderId' in data:
        return data
    else:
        raise Exception(f"Error placing order: {data}")

In [ ]:
# Calculate order size based on investment amount and current price
def calculate_order_size(investment_amount, current_price):
    return investment_amount / current_price

In [ ]:
# Implement the DCA strategy

#interval: investment are made at consistent, predetermined intervals (e.g. weekly, monthly)
#investment_amount: each investment is of the same dollar amount
#duration: the duration of the investment (e.g. 12 months)

def dca_investment(symbol, interval, investment_amount, duration):
    for _ in range(duration):
        current_price = fetch_market_price(symbol)
        if current_price:
            order_size = calculate_order_size(investment_amount, current_price)
            try:
                order = place_order(symbol, 'buy', order_size, current_price)
                print(f"Placed DCA order: {order}")
            except Exception as e:
                print(f"Error placing order: {e}")
        else:
            print("Failed to fetch the current price.")
        time.sleep(interval)

In [ ]:
# Example usage
symbol = 'BTCUSDT'
interval = 86400  # 24 hours in seconds
investment_amount = 100  # $100 investment per interval
duration = 7  # 7 days

dca_investment(symbol, interval, investment_amount, duration)